This iPython notebook describes how to use the ABS3D code for 3D correction of X-ray absorption in STEM-EDX tomography.

First, begin by importing the necessary packages. Note, as well as numpy and hyperspy, astra, scipy, scikit-image and matplotlib should also be installed in order to run this notebook.

In [ ]:
import hyperspy.api as hspy
import numpy as np
import matplotlib.pyplot as plt

#Import the relevant parts of the package
from edx_abs import abs_3d, mass_atten_dict, gpu_abs_3d
import edx_abs.tools_3d as t3d

The parameters of the data should be set up before importing the data. You must know the tilt angles at which the data was acquired, the elements in question and their densities.

In [ ]:
angles = np.arange(-90,91,2.5)
angles = np.delete(angles,16)

elements = ['Ni','Fe','Cr','O']
densities = {'Ni':8.9,'Fe':7.9,'Cr':7.2,'O':5}

It is possible to set the EDX detector parameters using the detectors class. If you choose not to set the detector parameters they will default to two detectors at an elevation of 15 degrees and azimuthal angles of 135 and 225 degrees.

In [ ]:
det = abs_3d.detectors(2,[{'elevation':15,'azimuth':135},{'elevation':15,'azimuth':225}])

Next, we import the data in to a dictionary indexed by each element.

In [ ]:
root_folder = 'datasets/'

#Import the tilt series data in to a dictionary
series = {}
for el in elements:
    filename = root_folder + el + '_series.tif'
    series[el] = hspy.load(filename).data

We can then reconstruct each dataset using the reconstruct function in tools3d.

In [ ]:
recons = {}
for el in elements:
    recons[el] = t3d.reconstruct(series[el],angles)
    if np.min(recons[el]) < 0:
        recons[el] = recons[el] - np.min(recons[el])

The same is done for the HAADF data.

In [ ]:
haadf_series = hspy.load(root_folder+'haadf_series.tif').data
haadf_recon = t3d.reconstruct(haadf_series,angles)

Next, inputting the total composition determined externally, use the function k_factor_matrix_dict to generate Cliff-Lorimer factors for each elemental pair that are stored in k_factors.

In [ ]:
esprit_weightpercent = {'Ni':67,'Fe':7,'Cr':12,'O':7}

k_factors = abs_3d.k_factor_matrix_dict(recons,esprit_weightpercent)

Using the Cliff-Lorimer factors generated in the previous step, calculate the composition in each voxel using the function composition_map. Ignore the divide by zero warning.

In [ ]:
comp_maps = abs_3d.composition_map(recons,k_factors,elements)

Calculate the 3D density distribution from the HAADF reconstruction using calibration coefficients determined from know phases within the sample.

In [ ]:
density = np.zeros(np.shape(haadf_recon))
density = 0.15*np.sqrt(haadf_recon)-5
density[haadf_recon<3000] = 0

Change the compositional maps so that every voxel in which the density is zero, the composition is also zero.

In [ ]:
norm_comp_maps = {}
for comp_map in comp_maps:
    norm_comp_maps[comp_map] = comp_maps[comp_map]
    norm_comp_maps[comp_map][density==0] = 0

Slices from the reconstructions can be visualised using matplotlib.

In [ ]:
fig, axes = plt.subplots(2,2)
for val,el in enumerate(elements):
    axes.flat[val].set_title(el)
    im = axes.flat[val].imshow(norm_comp_maps[el][:,88],clim=[0,1],cmap='plasma')
    axes.flat[val].xaxis.set_visible(False),axes.flat[val].yaxis.set_visible(False)
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
cbar_ax.set_title('Weight fraction')
fig.colorbar(im, cax=cbar_ax)

Calculate the 3D volume of mass attenuation values. First, generate a dictionary of elemental mass attenuation values. For each element, the weight fraction is multiplied by the elemental mass attenuation value and all elements are summed to give the final mass attenuation value.

In [ ]:
mass_atten_dic = mass_atten_dict.generate_mass_atten_dict()

#Mass attenuation maps
mass_atten = {}

for xray_peak in mass_atten_dic:
    mass_atten[xray_peak] = np.zeros(np.shape(haadf_recon))
    for element in norm_comp_maps:
        mass_atten[xray_peak] = mass_atten[xray_peak] + norm_comp_maps[element]*mass_atten_dic[xray_peak][element]

Using the function gpu_acf_series, calculate the X-ray absorption in each pixel in each image of the tilt series. This may take some time.

In [ ]:
#Calculate series of ACF projections
abs_series = {}
pix_size = np.float32(8*10**-7)

for el in ['Ni_Ka','O_Ka', 'Cr_Ka', 'Fe_Ka']:
    el_series = gpu_abs_3d.gpu_acf_series(density,mass_atten[el],recons[el.rpartition('_')[0]],angles,pix_size,detectors=det)
    
    abs_series[el] = el_series

Correct the initial projections with the X-ray absorption expected in each pixel.

In [ ]:
empty_proj = np.zeros((72,704,704))
series_abs_corr = {}

for el1 in series:
    for el2 in abs_series:
        if el1 == el2.rpartition('_')[0]:
            series_abs_corr[el1] = series[el1]/np.asarray(abs_series[el2][0])

Reconstruct the corrected projections to obtain an absorption corrected reconstruction. Recalculate the composition using the previously calculated Cliff-Lorimer factors to obtain a final corrected composition. Further iterations of the correction are possible but were found to have little effect on the final composition.

In [ ]:
abs_rec_1it = {}
for el in elements:
    abs_rec_1it[el] = t3d.reconstruct(series_abs_corr[el],angles)
    if np.min(abs_rec_1it[el]) < 0:
        abs_rec_1it[el] = abs_rec_1it[el] - np.min(abs_rec_1it[el])
    
abs_1it_comp_maps = abs_3d.composition_map(abs_rec_1it,k_factors,elements)

norm_comp_maps_1it = {}
for comp_map in abs_1it_comp_maps:
    norm_comp_maps_1it[comp_map] = abs_1it_comp_maps[comp_map]
    norm_comp_maps_1it[comp_map][density==0] = 0

Visualise the newly calculated composition.

In [ ]:
fig, axes = plt.subplots(2,2)
for val,el in enumerate(elements):
    axes.flat[val].set_title(el)
    im = axes.flat[val].imshow(norm_comp_maps_1it[el][:,88],clim=[0,1],cmap='plasma')
    axes.flat[val].xaxis.set_visible(False),axes.flat[val].yaxis.set_visible(False)
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
cbar_ax.set_title('Weight fraction')
fig.colorbar(im, cax=cbar_ax)

Visualise the difference in composition between corrected and uncorrected datasets.

In [ ]:
fig, axes = plt.subplots(2,2)
for val,el in enumerate(elements):
    axes.flat[val].set_title(el)
    im = axes.flat[val].imshow(norm_comp_maps_1it[el][:,88]-norm_comp_maps[el][:,88],clim=[-0.1,0.1],cmap='plasma')
    axes.flat[val].xaxis.set_visible(False),axes.flat[val].yaxis.set_visible(False)
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
cbar_ax.set_title('Weight fraction')
fig.colorbar(im, cax=cbar_ax)